In [1]:
import os
import csv
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.image as img
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Dataset

In [2]:
# Converting all the known Images to grey images
path = 'KnownImages'
myList = os.listdir(path)

for i in myList:
    openedImg = Image.open(f'{path}/{i}')
    studentNames = os.path.splitext(i)[0]
    newName = studentNames + 'grey'
    openedImg = openedImg.resize((64, 64)).convert('L')
    openedImg.save('GreyedImages/' + newName + '.png')

In [3]:
# Encoding thes images array and adding them to the dataset
path = 'GreyedImages'
myList = os.listdir(path)    
images_greyed = []
count = 0 

with open(r'datasets\olivetti_X.csv', 'a', newline='') as f:
    for i in myList:
        count = count + 1
        im = img.imread(f'{path}/{i}')
        image_reshape = (im.flatten().reshape(-1, 1).T)
        writer = csv.writer(f)        
        writer.writerows(image_reshape)
        
        lastRow = pd.read_csv("datasets\olivetti_y.csv").iloc[-1][0]
        data = [str(lastRow+1)]
        with open(r'datasets\olivetti_y.csv', 'a', newline='') as file:
            writer = csv.writer(file)  
            writer.writerow(data)
file.close()
f.close()

In [4]:
# Reading the updated datasets as array
data = np.loadtxt(open("datasets\olivetti_X.csv", "rb"), delimiter=",", skiprows=1)
target = np.loadtxt(open("datasets\olivetti_Y.csv", "rb"), delimiter=",", skiprows=1)
data

array([[0.45454547, 0.47107437, 0.51239669, ..., 0.15289256, 0.15289256,
        0.15289256],
       [0.31818181, 0.40082645, 0.49173555, ..., 0.14049587, 0.14876033,
        0.15289256],
       [0.19834711, 0.19421488, 0.19421488, ..., 0.75206614, 0.75206614,
        0.73966944],
       ...,
       [0.34901962, 0.36078432, 0.627451  , ..., 0.19607843, 0.46666667,
        0.69411767],
       [0.24313726, 0.25490198, 0.28627452, ..., 0.14901961, 0.14117648,
        0.12156863],
       [0.20392157, 0.40784314, 0.45490196, ..., 0.22352941, 0.21960784,
        0.20784314]])

# Data Anyalysis

In [5]:
print("There are {} images in the dataset".format(len(data)))
print("There are {} unique targets in the dataset".format(len(np.unique(target))))
print("There are {} input features".format(data.shape[1]))
print("Size of each image is 64 x 64")

There are 405 images in the dataset
There are 46 unique targets in the dataset
There are 4096 input features
Size of each image is 64 x 64


# Data Pre-processing 

# Spliting Dataset

In [6]:
X = data
y = target
Height = 64
Width = 64
# Split into a training set (75%) and a test set (25%) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# PCA

In [7]:
pca = PCA(n_components=150, whiten=True).fit(X_train)

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

print("Current shape of input data matrix: ", X_train_pca.shape)

Current shape of input data matrix:  (303, 150)


# KNeighborsClassifier

In [8]:
model = KNeighborsClassifier(n_neighbors = 5)
model.fit(X_train_pca, y_train)

KNeighborsClassifier()

In [9]:
accuracy = model.score(X_test_pca,y_test)
print("Testing Score = {:.3f}".format(accuracy))

Testing Score = 0.490


In [10]:
accuracy = model.score(X_train_pca,y_train)
print("Training Score = {:.3f}".format(accuracy))

Training Score = 0.640


# Confusion Matrix

In [11]:
y_test_pred = model.predict(X_test_pca)
cfm = confusion_matrix(y_test,y_test_pred)
cfm

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

# Evaluation Metrics

In [12]:
cReport = classification_report(y_test, y_test_pred)
print(cReport)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         3
         1.0       0.00      0.00      0.00         3
         2.0       0.00      0.00      0.00         2
         3.0       0.50      0.33      0.40         3
         4.0       0.50      1.00      0.67         4
         5.0       0.25      0.33      0.29         3
         6.0       0.00      0.00      0.00         1
         7.0       1.00      0.33      0.50         6
         8.0       0.33      1.00      0.50         2
         9.0       1.00      0.33      0.50         3
        10.0       1.00      0.67      0.80         3
        11.0       1.00      0.67      0.80         3
        12.0       0.00      0.00      0.00         2
        13.0       1.00      1.00      1.00         3
        14.0       0.18      1.00      0.30         3
        15.0       0.00      0.00      0.00         2
        16.0       0.00      0.00      0.00         1
        17.0       0.75    

C:\Users\Danyal\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Danyal\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Danyal\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Danyal\anaconda3\lib\site-packages\sklearn\metrics\

# Resetting Datasets

In [13]:
df = pd.read_csv('datasets/olivetti_X.csv')
df.drop(df.tail(count).index,inplace=True)
df.to_csv('datasets\olivetti_X.csv', index=False)

In [14]:
df = pd.read_csv('datasets/olivetti_y.csv')
df.drop(df.tail(count).index,inplace=True)
df.to_csv('datasets\olivetti_y.csv', index=False)